In [1]:
# ====== SECTION 1: ENVIRONMENT SETUP ======
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [2]:
# ====== SECTION 2: IMAGE TRANSFORMS ======
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # MobileNet needs 3 channels
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])


In [3]:
# ====== STEP 1: MOUNT GOOGLE DRIVE ======
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# ====== STEP 2: DATASET PATH ======
BASE_DIR = "/content/drive/MyDrive/Final_Dataset"

train_dir = BASE_DIR + "/train"
val_dir   = BASE_DIR + "/val"


In [5]:
!ls /content/drive/MyDrive/Final_Dataset


test  train  val


In [6]:
BASE_DIR = "/content/drive/MyDrive/Final_Dataset"

train_dir = f"{BASE_DIR}/train"
val_dir   = f"{BASE_DIR}/val"

print(train_dir)
print(val_dir)


/content/drive/MyDrive/Final_Dataset/train
/content/drive/MyDrive/Final_Dataset/val


In [7]:
!ls "$train_dir"



 Bridge		        CMP-Scratches_defect	  'LER_line width roughness'
 Clean		        Cracks			   Opens
 CMP-Corrosion_defect   Gap			   VIAS_Alignment_defect
 CMP-Dishing_defect    'LER_break defect'	  'VIAS_Incomplete Etch_Defect'
 CMP-Errosion_defect   'LER_bridge defect'	   VIAS_sidewall_defect
 CMP-Residue_defect    'LER_line edge roughness'   VIAS_Voids_defect


In [8]:
# ====== STEP 0: DATASET PATHS ======
BASE_DIR = "/content/drive/MyDrive/Final_Dataset"

train_dir = BASE_DIR + "/train"
val_dir   = BASE_DIR + "/val"

print(train_dir)
print(val_dir)


/content/drive/MyDrive/Final_Dataset/train
/content/drive/MyDrive/Final_Dataset/val


In [9]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch

# Image transforms (grayscale + MobileNet compatible)
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])
])

# Load datasets
train_data = datasets.ImageFolder(train_dir, transform=transform)
val_data   = datasets.ImageFolder(val_dir, transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_data, batch_size=32, shuffle=False)

main_classes = train_data.classes
num_main_classes = len(main_classes)

print("Main Classes:", main_classes)


Main Classes: ['Bridge', 'CMP-Corrosion_defect', 'CMP-Dishing_defect', 'CMP-Errosion_defect', 'CMP-Residue_defect', 'CMP-Scratches_defect', 'Clean', 'Cracks', 'Gap', 'LER_break defect', 'LER_bridge defect', 'LER_line edge roughness', 'LER_line width roughness', 'Opens', 'VIAS_Alignment_defect', 'VIAS_Incomplete Etch_Defect', 'VIAS_Voids_defect', 'VIAS_sidewall_defect']


In [10]:
# ====== STAGE 4: MAIN CNN MODEL ======
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

main_model = models.mobilenet_v2(pretrained=True)

for param in main_model.features.parameters():
    param.requires_grad = False

main_model.classifier = nn.Sequential(
    nn.Linear(main_model.last_channel, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, num_main_classes)
)

main_model = main_model.to(device)



/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(main_model.classifier.parameters(), lr=0.001)

epochs = 15

for epoch in range(epochs):
    main_model.train()
    running_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = main_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/15] Loss: 1.9712
Epoch [2/15] Loss: 1.2215
Epoch [3/15] Loss: 0.9562
Epoch [4/15] Loss: 0.8373
Epoch [5/15] Loss: 0.7281
Epoch [6/15] Loss: 0.6629
Epoch [7/15] Loss: 0.5928
Epoch [8/15] Loss: 0.5308
Epoch [9/15] Loss: 0.5447
Epoch [10/15] Loss: 0.4521
Epoch [11/15] Loss: 0.4519
Epoch [12/15] Loss: 0.4266
Epoch [13/15] Loss: 0.3519
Epoch [14/15] Loss: 0.4029
Epoch [15/15] Loss: 0.3634


In [36]:
torch.save(main_model.state_dict(), "/content/drive/MyDrive/main_model.pth")
print("Model saved successfully!")


Model saved successfully!


In [12]:
# ====== STAGE 6: FEATURE EXTRACTOR ======
def extract_features(model, image_tensor):
    model.eval()
    with torch.no_grad():
        x = image_tensor
        x = model.features(x)
        x = x.mean([2, 3])  # Global Average Pooling
    return x


In [13]:
print(extract_features)


<function extract_features at 0x7cc8d17ee340>


In [14]:
# ====== STAGE 7: CLASS CENTROIDS ======
from collections import defaultdict

def compute_class_centroids(model, dataloader, class_names):
    centroids = defaultdict(list)

    for images, labels in dataloader:
        images = images.to(device)
        features = extract_features(model, images)

        for i, label in enumerate(labels):
            class_name = class_names[label.item()]
            centroids[class_name].append(features[i].cpu())

    # Mean feature per class
    for cls in centroids:
        centroids[cls] = torch.stack(centroids[cls]).mean(dim=0)

    return centroids


In [15]:
# ====== COMPUTE CLASS CENTROIDS (MANDATORY STEP) ======
class_centroids = compute_class_centroids(
    main_model,
    train_loader,
    main_classes
)


In [16]:
# ====== SECTION 8: AUTOMATIC THRESHOLD CALIBRATION ======
import numpy as np

def calibrate_threshold(model, dataloader, centroids, percentile=98):
    """
    Calculates the distance threshold based on the Validation Set.
    We set the limit at the 98th percentile of known defect distances.
    """
    model.eval()
    distances = []

    print("⚙️ Calibrating threshold using Validation Set... (This takes a moment)")

    with torch.no_grad():
        for images, labels in dataloader:
            images = images.to(device)
            # 1. Extract features
            features = extract_features(model, images)

            for i, label in enumerate(labels):
                # 2. Get the correct class name for this image
                class_name = main_classes[label.item()]

                # 3. Get the centroid for that class
                centroid = centroids[class_name].to(device)
                feature = features[i].to(device)

                # 4. Measure distance (How far is this image from its "home"?)
                dist = torch.norm(feature - centroid).item()
                distances.append(dist)

    # Convert to numpy to find statistics
    distances = np.array(distances)

    # Set threshold to cover 98% of valid cases (ignores top 2% outliers)
    calculated_threshold = np.percentile(distances, percentile)

    print(f"📊 Statistics:")
    print(f"   - Min Distance: {distances.min():.4f}")
    print(f"   - Avg Distance: {distances.mean():.4f}")
    print(f"   - Max Distance: {distances.max():.4f}")
    print(f"✅ NEW THRESHOLD SET: {calculated_threshold:.4f}")

    return calculated_threshold

# ---- EXECUTE CALIBRATION ----
# This will calculate the magic number for you
CALIBRATED_THRESHOLD = calibrate_threshold(main_model, val_loader, class_centroids)

⚙️ Calibrating threshold using Validation Set... (This takes a moment)
📊 Statistics:
   - Min Distance: 8.7674
   - Avg Distance: 17.1797
   - Max Distance: 28.6449
✅ NEW THRESHOLD SET: 27.3183


In [17]:
# ====== UNKNOWN DEFECT DETECTION FUNCTION (MANDATORY) ======
import torch

def detect_unknown(feature, class_centroids, threshold=1.2):
    """
    feature: Tensor of shape [feature_dim]
    class_centroids: dict {class_name: centroid_tensor}
    threshold: distance threshold for unknown detection
    """

    min_dist = float("inf")
    predicted_class = None

    for cls, centroid in class_centroids.items():
        dist = torch.norm(feature - centroid)
        if dist < min_dist:
            min_dist = dist
            predicted_class = cls

    is_unknown = min_dist > threshold
    return is_unknown, min_dist.item()


In [18]:
print(detect_unknown)


<function detect_unknown at 0x7cc8d17ef9c0>


In [19]:
print(main_model)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [20]:
root_cause_map = {

# CMP
"CMP_Scratches": {
    "cause": "Abrasive particle contamination during polishing",
    "action": "Inspect slurry purity and pad condition"
},
"CMP_Residue": {
    "cause": "Incomplete post-CMP cleaning",
    "action": "Improve cleaning chemistry"
},
"CMP_Erosion": {
    "cause": "Excessive polish pressure",
    "action": "Optimize CMP pressure"
},
"CMP_Dishing": {
    "cause": "Over-polishing of metal regions",
    "action": "Adjust polish selectivity"
},
"CMP_Corrosion": {
    "cause": "Chemical reaction after CMP",
    "action": "Control slurry chemistry"
},

# VIAS
"VIAS_Alignment_defect": {
    "cause": "Lithography overlay error",
    "action": "Recalibrate alignment system"
},
"VIAS_Sidewall_defect": {
    "cause": "Plasma etch instability",
    "action": "Optimize etch recipe"
},
"VIAS_Incomplete_Etch_defect": {
    "cause": "Insufficient etch time",
    "action": "Increase etch duration"
},
"VIAS_Voids_defect": {
    "cause": "Gas entrapment during fill",
    "action": "Improve deposition coverage"
},

# LER
"LER_line_width_roughness": {
    "cause": "Resist thickness variation",
    "action": "Improve resist coating"
},
"LER_line_edge_roughness": {
    "cause": "Lithography shot noise",
    "action": "Optimize exposure dose"
},
"LER_bridge_defect": {
    "cause": "Resist swelling",
    "action": "Tune exposure and bake"
},
"LER_break_defect": {
    "cause": "Underexposure",
    "action": "Adjust exposure and develop time"
}
}


In [21]:
print(main_model)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [22]:
print(extract_features)


<function extract_features at 0x7cc8d17ee340>


In [23]:
print(detect_unknown)


<function detect_unknown at 0x7cc8d17ef9c0>


In [24]:
print(class_centroids.keys())


dict_keys(['VIAS_Voids_defect', 'Bridge', 'Clean', 'VIAS_Alignment_defect', 'VIAS_sidewall_defect', 'Cracks', 'Gap', 'VIAS_Incomplete Etch_Defect', 'CMP-Errosion_defect', 'CMP-Scratches_defect', 'Opens', 'LER_break defect', 'CMP-Residue_defect', 'CMP-Dishing_defect', 'CMP-Corrosion_defect', 'LER_bridge defect', 'LER_line width roughness', 'LER_line edge roughness'])


In [25]:
print(root_cause_map["CMP_Scratches"])


{'cause': 'Abrasive particle contamination during polishing', 'action': 'Inspect slurry purity and pad condition'}


In [26]:
# ====== SECTION 9: FINAL INFERENCE PIPELINE ======

# 1. Updated Unknown Detector using the New Threshold
def detect_unknown(feature, class_centroids, threshold):
    feature = feature.cpu() # Ensure CPU for comparison
    min_dist = float("inf")
    predicted_class = None

    for cls, centroid in class_centroids.items():
        # Calculate distance to every known class
        dist = torch.norm(feature - centroid)
        if dist < min_dist:
            min_dist = dist
            predicted_class = cls

    # Check if the closest distance is still too far
    is_unknown = min_dist > threshold
    return is_unknown, min_dist.item()

# 2. Updated Inference Function
def final_inference_with_unknown(image_tensor, threshold=CALIBRATED_THRESHOLD):

    # ---- Feature extraction ----
    # Get the embedding vector (the "DNA" of the image)
    feature = extract_features(main_model, image_tensor).squeeze()

    # ---- Unknown detection ----
    is_unknown, dist = detect_unknown(feature, class_centroids, threshold)

    # DEBUG PRINT: This tells you exactly what the model sees
    print(f"🔍 DEBUG: Distance = {dist:.4f} | Threshold = {threshold:.4f}")

    if is_unknown:
        return {
            "Defect Type": "Unknown",
            "Confidence": "Low",
            "Reason": f"Distance ({dist:.2f}) exceeds threshold",
            "Action Suggested": "Send image for human review"
        }

    # ---- Known defect classification ----
    main_model.eval()
    with torch.no_grad():
        output = main_model(image_tensor)
        prob = torch.softmax(output, dim=1)
        conf, idx = torch.max(prob, 1)

    defect_type = main_classes[idx.item()]

    result = {
        "Defect Type": defect_type,
        "Confidence": f"{conf.item()*100:.1f}%"
    }

    # Add Root Cause Info
    if defect_type in root_cause_map:
        info = root_cause_map[defect_type]
    elif defect_type in ["CMP", "VIAS", "LER"]:
        # Fallback if subtype logic isn't ready
        result["Note"] = "Subtype classification skipped"
        return result
    else:
        # Generic fallback
        result["Probable Cause"] = "General Process Drift"
        result["Action Suggested"] = "Check Tool Logs"
        return result

    result["Probable Cause"] = info["cause"]
    result["Action Suggested"] = info["action"]

    return result

In [27]:
print(final_inference_with_unknown)


<function final_inference_with_unknown at 0x7cc8d166c4a0>


In [28]:
inference_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # 🔥 KEY FIX
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])


In [29]:
from PIL import Image

# Load your test image
img_path = "/content/drive/MyDrive/Final_Dataset/test/VIAS_Alignment_defect/1_aug_8.png"
img = Image.open(img_path).convert("L")

# Preprocess
test_image_tensor = inference_transform(img).unsqueeze(0).to(device)

# Run Inference
result = final_inference_with_unknown(test_image_tensor)
print("\n🚀 FINAL RESULT:")
print(result)

🔍 DEBUG: Distance = 19.3085 | Threshold = 27.3183

🚀 FINAL RESULT:
{'Defect Type': 'VIAS_Alignment_defect', 'Confidence': '99.9%', 'Probable Cause': 'Lithography overlay error', 'Action Suggested': 'Recalibrate alignment system'}


In [30]:
main_model.eval()


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [31]:
dummy_input = torch.randn(1, 3, 224, 224).to(device)


In [32]:
!pip install onnx onnxruntime onnxscript


In [37]:
!ls /content/drive/MyDrive/main_model.pth


/content/drive/MyDrive/main_model.pth


In [38]:
import torch
import torch.nn as nn
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

main_classes = ['Bridge','CMP','Clean','Cracks','Gap','LER','Opens','Other','VIAS']
num_main_classes = len(main_classes)

main_model = models.mobilenet_v2(pretrained=False)

for param in main_model.features.parameters():
    param.requires_grad = False

main_model.classifier = nn.Sequential(
    nn.Linear(main_model.last_channel, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, num_main_classes)
)

main_model = main_model.to(device)
main_model.eval()


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [40]:
!ls /content/drive/MyDrive/Final_Dataset/train


 Bridge		        CMP-Scratches_defect	  'LER_line width roughness'
 Clean		        Cracks			   Opens
 CMP-Corrosion_defect   Gap			   VIAS_Alignment_defect
 CMP-Dishing_defect    'LER_break defect'	  'VIAS_Incomplete Etch_Defect'
 CMP-Errosion_defect   'LER_bridge defect'	   VIAS_sidewall_defect
 CMP-Residue_defect    'LER_line edge roughness'   VIAS_Voids_defect


In [41]:
main_classes = [
    'Bridge',
    'Clean',
    'Opens',
    'Gap',
    'Cracks',
    'LER_line_edge_roughness',
    'LER_line_width_roughness',
    'LER_bridge_defect',
    'LER_break_defect',
    'CMP_Scratches',
    'CMP_Residue',
    'CMP_Erosion',
    'CMP_Corrosion',
    'CMP_Dishing',
    'VIAS_Alignment_defect',
    'VIAS_sidewall_defect',
    'VIAS_Incomplete_Etch_defect',
    'VIAS_Voids_defect'
]

num_main_classes = len(main_classes)
print(num_main_classes)


18


In [42]:
main_model = models.mobilenet_v2(pretrained=False)

for param in main_model.features.parameters():
    param.requires_grad = False

main_model.classifier = nn.Sequential(
    nn.Linear(main_model.last_channel, 128),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(128, num_main_classes)
)

main_model = main_model.to(device)
main_model.eval()


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [43]:
main_model.load_state_dict(
    torch.load("/content/drive/MyDrive/main_model.pth", map_location=device)
)
print("Loaded successfully")


Loaded successfully


In [44]:
dummy_input = torch.randn(1, 3, 224, 224).to(device)

torch.onnx.export(
    main_model,
    dummy_input,
    "semiconductor_defect_model.onnx",
    export_params=True,
    opset_version=11,
    do_constant_folding=True,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={
        "input": {0: "batch"},
        "output": {0: "batch"}
    }
)

print("✅ ONNX model exported successfully!")


/tmp/ipython-input-3461244328.py:3: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(
W0207 13:49:24.456000 16198 torch/onnx/_internal/exporter/_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 11 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features
W0207 13:49:26.239000 16198 torch/onnx/_internal/exporter/_schemas.py:455] Missing annotation for parameter 'input' from (input, boxes, output_size: 'Sequence[int]', spatial_scale: 'float' = 1.0, sampling_ratio: 'int' = -1, a

[torch.onnx] Obtain model graph for `MobileNetV2([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `MobileNetV2([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
             ^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnxscript/version_converter/__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/onnx/version_converter.py", line 39, in convert_version
    converted_model_str = C.convert_version(model_str, target_version)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: /github/workspace/onnx/version_converter/adapters/axes_input_to_attribute.h:65: adapt: Asserti

[torch.onnx] Translate the graph into ONNX... ✅
Applied 105 of general pattern rewrite rules.
✅ ONNX model exported successfully!


In [45]:
!ls -lh semiconductor_defect_model.onnx


-rw-r--r-- 1 root root 251K Feb  7 13:49 semiconductor_defect_model.onnx
